[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Optimization Methods

## SVD & Linear Least Squares - Weighted Least Squares for Trigonometric Polynomials

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 12/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0012LinearFitL1.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Miscellaneous
from platform import python_version
import random

# Typing
from typing import Callable, List, Optional, Tuple, Union

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 640 # 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions


In [ ]:
# Parameters

# Data
numSamples  = 500
dataFreq    = 0.05 #<! Keep below 0.5

vX = 0.5 + np.random.rand(2) #<! Amplitude

𝜋 = np.pi

## Weighted Least Squares

The classic [Weighted Least Squares](https://en.wikipedia.org/wiki/Weighted_least_squares) model is given by:

$$ \arg \min_{\boldsymbol{x}} \frac{1}{2} \sum_{i} {w}_{i} {\left( \boldsymbol{a}_{i}^{T} \boldsymbol{x} - {b}_{i} \right)}^{2} $$

The ${w}_{i} \geq 0$ coefficient is a prior knowledge about the quality of the $i$ -th sample.  
The higher the value of ${w}_{i}$ the smaller ${\left( \boldsymbol{a}^{T} \hat{\boldsymbol{x}} - {b}_{i} \right)}^{2}$ will be.  
Namely, the value of ${w}_{i}$ sets the importance and quality of the sample ${b}_{i}$.

The model can be generalized by a Symmetric Positive Semi Definite (SPSD) matrix $\boldsymbol{W}$:

$$ \arg \min_{\boldsymbol{x}} {\left( \boldsymbol{A} \boldsymbol{x} - \boldsymbol{b} \right)}^{T} \boldsymbol{W} {\left( \boldsymbol{A} \boldsymbol{x} - \boldsymbol{b} \right)} $$

Where $\boldsymbol{a}_{i}$ is the $i$ -th row of $\boldsymbol{A}$.


* <font color='brown'>(**#**)</font> In the context of Estimation, one can model the data by $\boldsymbol{b} \sim \mathcal{N} \left( \boldsymbol{A} \boldsymbol{x}, \boldsymbol{C} \right)$ where $\boldsymbol{W} = \boldsymbol{C}^{-1}$.



## Generate Data


The data model is a [Trigonometric Polynomial](https://en.wikipedia.org/wiki/Trigonometric_polynomial).

In [ ]:
# Generate / Load the Data

# Model
vT = np.arange(numSamples)
mA = np.column_stack((np.sin(2 * 𝜋 * dataFreq * vT), np.cos(2 * 𝜋 * dataFreq * vT)))
vA = mA @ vX

# Weights
vW = 0.75 * np.random.rand(numSamples)
vW = np.sort(vW)

# Generate a random orthonormal matrix
mT = np.random.rand(numSamples, numSamples)
mQ, mR = sp.linalg.qr(mT)

# Weights
# mW      = (mQ.T * vW) @ mQ #<! mQ.T @ np.diag(vW) @ mQ
mW      = mQ.T @ np.diag(vW) @ mQ
mWSqrt  = sp.linalg.cholesky(mW)

# Colored Noise
mC = np.linalg.inv(mW)
mCSqrt = sp.linalg.cholesky(mC, lower = True)

vN = mCSqrt @ np.random.randn(numSamples)
vB = vA + vN #<! Data Samples

In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (10, 6))
hA.plot(range(numSamples), vA, linewidth = 2, label = 'Data Model')
hA.scatter(range(numSamples), vB, s = 20, c = vW, label = 'Data Samples')
hA.set(xlabel = 'Sample Index', ylabel = 'Sample Value', title = 'Model and Noisy Samples')

hA.legend();

* <font color='red'>(**?**)</font> Explain the noise model.  
Remember that for a centered random vector, $\boldsymbol{x}$ the covariance matrix is given by: $\boldsymbol{C} = \mathbb{E} \left[ \boldsymbol{x} \boldsymbol{x}^{T} \right]$.

## The Least Squares Solution

This section calculates the solution for the Ordinary Least Squares model.

In [ ]:
# Least Squares Solution

#===========================Fill This===========================#
# 1. Calculate the least squares solution for the amplitude estimation.
# 2. The given data: `mA`, 'vB`. Name the solution `vXLS`.
# !! You may find `np.linalg.lstsq()` useful.

vXLS, *_ = np.linalg.lstsq(mA, vB, rcond = None)
#===============================================================#

* <font color='red'>(**?**)</font> For what kind of $\boldsymbol{C}$ the above would be optimal?

## The Weighted Least Squares Solution

Given the model:

$$ \arg \min_{\boldsymbol{x}} {\left( \boldsymbol{A} \boldsymbol{x} - \boldsymbol{b} \right)}^{T} \boldsymbol{W} {\left( \boldsymbol{A} \boldsymbol{x} - \boldsymbol{b} \right)} $$

The solution could be achieved either by "Whitening" the data and solving a regular LS or by solving the weighted model.

In [ ]:
# Weighted Least Squares Solution

#===========================Fill This===========================#
# 1. Calculate the weighted least squares solution for the amplitude estimation.
# 2. The given data: `mA`, 'vB`, `mW`, `mWSqrt`. Name the solution `vXWLS`.
# !! You may chose to solve either by whitening or the direct method.
# !! You may find `np.linalg.lstsq()` useful.

# Whitening
vYY = mWSqrt @ vB
mAA = mWSqrt @ mA
vXWLS, *_ = np.linalg.lstsq(mAA, vYY, rcond = None)

# Direct
vXWLS, *_ = np.linalg.lstsq(mA.T @ mW @ mA, mA.T @ mW @ vB, rcond = None)
#===============================================================#

* <font color='brown'>(**#**)</font> NumPy / SciPy does not have a direct solver of the weighted model.
* <font color='brown'>(**#**)</font> For the diagonal case one can solve the model using SciKit Learn's [`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) class.
* <font color='red'>(**?**)</font> Which approach is faster?

## Display Results

In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (15, 6))
hA.plot(range(numSamples), vA, linewidth = 2, label = 'Data Model')
hA.scatter(range(numSamples), vB, s = 20, c = vW, label = 'Data Samples')
hA.plot(range(numSamples), mA @ vXLS, linewidth = 2, label = 'LS Model')
hA.plot(range(numSamples), mA @ vXWLS, linewidth = 2, label = 'Weighted LS Model')
hA.set_ylim((-2, 2))
hA.set(xlabel = 'Sample Index', ylabel = 'Sample Value', title = 'Model and Noisy Samples')

hA.legend();

In [ ]:
print(f'LS Estimator L2 Norm Error : {np.linalg.norm(vXLS - vX):0.4f}')
print(f'WLS Estimator L2 Norm Error: {np.linalg.norm(vXWLS - vX):0.4f}')
print('')
print(f'LS Estimator RMSE          : {np.linalg.norm(mA @ vXLS - vB):0.4f}')
print(f'WLS Estimator RMSE         : {np.linalg.norm(mA @ vXWLS - vB):0.4f}')

* <font color='red'>(**?**)</font> How come the LS model gets a better RMSE for this model?
* <font color='red'>(**?**)</font> Which model better optimize the Estimator value?